In [ ]:
!pip install rasterio
import numpy as np
import rasterio
import matplotlib.pyplot as plt
from skimage.filters import threshold_otsu

# -------------------------
# Helper Functions
# -------------------------
def read_multispectral_tif(filepath):
    """
    Reads a multispectral .tif file and returns a numpy array of shape (rows, cols, bands).
    """
    with rasterio.open(filepath) as src:
        img = src.read()  # shape: (bands, rows, cols)
        img = np.transpose(img, (1, 2, 0))  # shape: (rows, cols, bands)
    return img

def compute_cvm(img1, img2):
    """
    Computes the Change Vector Magnitude (CVM) between two multispectral images.
    Assumes both images have identical shape (rows, cols, bands).
    """
    diff = img2.astype(np.float32) - img1.astype(np.float32)
    cvm = np.sqrt(np.sum(diff**2, axis=2))
    return cvm

def spectral_angle(pixel, ref):
    """
    Computes the spectral angle (in radians) between a pixel spectrum and a reference spectrum.
    Both inputs are 1-D arrays.
    """
    norm_pixel = np.linalg.norm(pixel)
    norm_ref = np.linalg.norm(ref)
    if norm_pixel == 0 or norm_ref == 0:
        return np.pi / 2  # Maximum angle if either vector is zero.
    cos_angle = np.clip(np.dot(pixel, ref) / (norm_pixel * norm_ref), -1.0, 1.0)
    return np.arccos(cos_angle)

def compute_sam_map(image, ref_spectrum):
    """
    Computes a SAM map for a multispectral image (rows x cols x bands)
    relative to the provided reference spectrum (length = bands).
    Returns an array (rows x cols) with spectral angles in radians.
    """
    rows, cols, bands = image.shape
    sam_map = np.zeros((rows, cols), dtype=np.float32)
    # Loop over each pixel (vectorized implementations are possible for speed)
    for i in range(rows):
        for j in range(cols):
            sam_map[i, j] = spectral_angle(image[i, j, :].astype(np.float32), ref_spectrum)
    return sam_map

def apply_otsu_threshold(map_array):
    """
    Applies Otsu thresholding to a 2D array and returns a binary mask and the threshold value.
    Also plots the histogram of pixel values with the calculated threshold.
    """

    # Compute Otsu's threshold
    thresh = threshold_otsu(map_array)
    binary_mask = map_array > thresh

    # Plot histogram with threshold line
    plt.figure(figsize=(8, 5))
    plt.hist(map_array.ravel(), bins=256, color='gray', alpha=0.7, density=True)
    plt.axvline(thresh, color='red', linestyle='dashed', linewidth=2, label=f'Threshold = {thresh:.2f}')
    plt.title("Histogram with Otsu's Threshold")
    plt.xlabel("Pixel Intensity")
    plt.ylabel("Frequency")
    plt.legend()
    plt.show()

    return binary_mask.astype(np.uint8), thresh

# -------------------------
# Main Pipeline
# -------------------------
# Define file paths for two Sentinel-2 images (13 bands each)
image1_path = "/content/preprocessed_stacked_1.tif"  # Replace with your actual file path
image2_path = "/content/preprocessed_stacked_2.tif"  # Replace with your actual file path

# Read the two multispectral images (assumed to be co-registered)
img1 = read_multispectral_tif(image1_path)  # shape: (rows, cols, 13)
img2 = read_multispectral_tif(image2_path)

# Compute the global Change Vector Magnitude (CVM) image
cvm = compute_cvm(img1, img2)

# -------------------------
# Define refined reference spectra for Sentinel-2 (13-band)
# Sentinel-2 bands order (common):
# B1: Coastal aerosol, B2: Blue, B3: Green, B4: Red, B5: Red Edge 1,
# B6: Red Edge 2, B7: Red Edge 3, B8: NIR, B8A: Narrow NIR,
# B9: Water vapor, B10: Cirrus, B11: SWIR 1, B12: SWIR 2.
#
# These reference values are chosen based on typical reflectance ranges:
ref_water = np.array([0.01, 0.01, 0.01, 0.005, 0.005, 0.005, 0.005, 0.01, 0.01, 0.005, 0.005, 0.005, 0.005])
ref_veg   = np.array([0.05, 0.1, 0.15, 0.1, 0.2, 0.25, 0.3, 0.6, 0.65, 0.1, 0.05, 0.03, 0.02])
ref_urban = np.array([0.15, 0.2, 0.25, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.25, 0.2, 0.2, 0.2])

# -------------------------
# Compute SAM maps for each target class using one representative image (use img2)
sam_water = compute_sam_map(img2, ref_water)
sam_veg   = compute_sam_map(img2, ref_veg)
sam_urban = compute_sam_map(img2, ref_urban)

# -------------------------
# Generate binary masks using Otsu thresholding for both CVM and each SAM map
mask_cvm, thresh_cvm = apply_otsu_threshold(cvm)
mask_sam_water, thresh_sam_water = apply_otsu_threshold(sam_water)
mask_sam_veg,   thresh_sam_veg   = apply_otsu_threshold(sam_veg)
mask_sam_urban, thresh_sam_urban = apply_otsu_threshold(sam_urban)

# -------------------------
# Create class-specific masks by combining (logical OR) the global CVM mask with each SAM mask.
mask_combined_water = np.logical_or(mask_cvm, mask_sam_water)
mask_combined_veg   = np.logical_or(mask_cvm, mask_sam_veg)
mask_combined_urban = np.logical_or(mask_cvm, mask_sam_urban)

# Compute class-specific CVM values by masking the global CVM image.
cvm_water = np.where(mask_combined_water, cvm, np.nan)
cvm_veg   = np.where(mask_combined_veg, cvm, np.nan)
cvm_urban = np.where(mask_combined_urban, cvm, np.nan)

# -------------------------
# Plotting: Create two sets of plots.
# (A) Contour maps for SAM and CVM for each class.
# (B) Overlay of the binary masks on the CVM image.

fig, axs = plt.subplots(3, 2, figsize=(14, 18))

def plot_contour(ax, data, title, cmap='jet'):
    cs = ax.contourf(data, levels=20, cmap=cmap)
    ax.set_title(title)
    plt.colorbar(cs, ax=ax)

# (A) Plot class-specific SAM contours (left column) and CVM contours (right column)
plot_contour(axs[0, 0], sam_water, "SAM Contours - Water")
plot_contour(axs[0, 1], cvm_water, "CVM Contours - Water")

plot_contour(axs[1, 0], sam_veg, "SAM Contours - Vegetation")
plot_contour(axs[1, 1], cvm_veg, "CVM Contours - Vegetation")

plot_contour(axs[2, 0], sam_urban, "SAM Contours - Urban")
plot_contour(axs[2, 1], cvm_urban, "CVM Contours - Urban")

plt.tight_layout()
plt.show()

# -------------------------
# (B) Plot overlay of the binary masks on the global CVM image.
# We create one figure with three subplots: one for each class.
fig2, axs2 = plt.subplots(1, 3, figsize=(18, 6))

def plot_mask_overlay(ax, base, mask, title):
    # Plot the base image (CVM)
    cs = ax.contourf(base, levels=20, cmap='viridis', alpha=0.8)
    # Overlay the binary mask as contour lines (or semi-transparent mask)
    ax.contour(mask, levels=[0.5], colors='red', linewidths=2)
    ax.set_title(title)
    plt.colorbar(cs, ax=ax)

plot_mask_overlay(axs2[0], cvm, mask_combined_water, "Water Mask Overlay on CVM")
plot_mask_overlay(axs2[1], cvm, mask_combined_veg,   "Vegetation Mask Overlay on CVM")
plot_mask_overlay(axs2[2], cvm, mask_combined_urban, "Urban Mask Overlay on CVM")

plt.tight_layout()
plt.show()
